In [1]:
import pandas as pd
import nltk
import pickle
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import string
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import plot_confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

import matplotlib.pyplot as plt
import seaborn as sns

stopwords = stopwords.words('english')

In [2]:
df = pd.read_csv('data/IMDB Dataset.csv')
print(df.shape)
df.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
print(df.isnull().sum())
print(df.duplicated().sum())

review       0
sentiment    0
dtype: int64
418


In [4]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [5]:
df.drop_duplicates(subset='review', inplace=True)

In [6]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

Feature Engineering

In [7]:
def preprocess(document):
    
    # convert to lower case
    document = document.lower()

    # tokenize document
    tk = RegexpTokenizer(r'[a-zA-Z]+')
    tokens = [token for token in tk.tokenize(document)]
    tokens = [token for token in tokens if token != 'br']

    # remove stop words
    tokens = [token for token in tokens if token not in stopwords]

    # stemmer
    porter = PorterStemmer()
    tokens = [porter.stem(token) for token in tokens]

    # remove words with length 1
    tokens = [token for token in tokens if len(token) > 1]
    
    return ' '.join(tokens)

In [8]:
df['clean_review'] = df['review'].apply(preprocess)
df.drop(['review'], axis=1, inplace=True)

In [9]:
df.head()

,sentiment,clean_review
0,positive,one review mention watch oz episod hook right ...
1,positive,wonder littl product film techniqu unassum old...
2,positive,thought wonder way spend time hot summer weeke...
3,negative,basic famili littl boy jake think zombi closet...
4,positive,petter mattei love time money visual stun film...


In [10]:
df['clean_review'][0]

'one review mention watch oz episod hook right exactli happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch darker side'

In [11]:
df = pd.get_dummies(df, columns=['sentiment'], drop_first=True)

In [12]:
X = df['clean_review'].tolist()
y = df['sentiment_positive'].tolist()

In [13]:
vectorizer = TfidfVectorizer(ngram_range=(1,3))

In [14]:
X = vectorizer.fit_transform(X)

In [15]:
X.shape

(49582, 7908403)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## MultinomialNB

In [17]:
model = MultinomialNB()

distributions = {'alpha': np.linspace(0,1,100)}
clf = RandomizedSearchCV(model, distributions, cv=15, verbose=100, n_jobs=11)

clf.fit(X_train, y_train)

Fitting 15 folds for each of 10 candidates, totalling 150 fits
[Parallel(n_jobs=11)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=11)]: Done   1 tasks      | elapsed:    2.6s
[Parallel(n_jobs=11)]: Done   2 tasks      | elapsed:    2.6s
[Parallel(n_jobs=11)]: Done   3 tasks      | elapsed:    2.6s
[Parallel(n_jobs=11)]: Done   4 tasks      | elapsed:    2.6s
[Parallel(n_jobs=11)]: Done   5 tasks      | elapsed:    2.7s
[Parallel(n_jobs=11)]: Done   6 tasks      | elapsed:    2.7s
[Parallel(n_jobs=11)]: Done   7 tasks      | elapsed:    2.7s
[Parallel(n_jobs=11)]: Done   8 tasks      | elapsed:    2.7s
[Parallel(n_jobs=11)]: Done   9 tasks      | elapsed:    2.8s
[Parallel(n_jobs=11)]: Done  10 tasks      | elapsed:    2.8s
[Parallel(n_jobs=11)]: Done  11 tasks      | elapsed:    2.8s
[Parallel(n_jobs=11)]: Done  12 tasks      | elapsed:    4.2s
[Parallel(n_jobs=11)]: Done  13 tasks      | elapsed:    4.3s
[Parallel(n_jobs=11)]: Done  14 tasks      | elapsed: 

[Parallel(n_jobs=11)]: Done 133 out of 150 | elapsed:   21.9s remaining:    2.7s
[Parallel(n_jobs=11)]: Done 135 out of 150 | elapsed:   22.3s remaining:    2.4s
[Parallel(n_jobs=11)]: Done 137 out of 150 | elapsed:   22.4s remaining:    2.0s
[Parallel(n_jobs=11)]: Done 139 out of 150 | elapsed:   22.5s remaining:    1.7s
[Parallel(n_jobs=11)]: Done 141 out of 150 | elapsed:   22.8s remaining:    1.4s
[Parallel(n_jobs=11)]: Done 143 out of 150 | elapsed:   23.0s remaining:    1.0s
[Parallel(n_jobs=11)]: Done 145 out of 150 | elapsed:   23.5s remaining:    0.7s
[Parallel(n_jobs=11)]: Done 147 out of 150 | elapsed:   23.7s remaining:    0.4s
[Parallel(n_jobs=11)]: Done 150 out of 150 | elapsed:   23.8s finished


RandomizedSearchCV(cv=15, estimator=MultinomialNB(), n_jobs=11,
                   param_distributions={'alpha': array([0.        , 0.01010101, 0.02020202, 0.03030303, 0.04040404,
       0.05050505, 0.06060606, 0.07070707, 0.08080808, 0.09090909,
       0.1010101 , 0.11111111, 0.12121212, 0.13131313, 0.14141414,
       0.15151515, 0.16161616, 0.17171717, 0.18181818, 0.19191919,
       0.2020202 , 0.21212121, 0.22222222, 0.23232323...
       0.65656566, 0.66666667, 0.67676768, 0.68686869, 0.6969697 ,
       0.70707071, 0.71717172, 0.72727273, 0.73737374, 0.74747475,
       0.75757576, 0.76767677, 0.77777778, 0.78787879, 0.7979798 ,
       0.80808081, 0.81818182, 0.82828283, 0.83838384, 0.84848485,
       0.85858586, 0.86868687, 0.87878788, 0.88888889, 0.8989899 ,
       0.90909091, 0.91919192, 0.92929293, 0.93939394, 0.94949495,
       0.95959596, 0.96969697, 0.97979798, 0.98989899, 1.        ])},
                   verbose=100)

In [18]:
print(clf.best_score_)

accuracy = clf.score(X_test, y_test)
print(accuracy)

0.8905333234191956
0.8883735000504185


In [19]:
clf.best_params_

{'alpha': 0.23232323232323235}

In [20]:
model = clf.best_estimator_

In [21]:
pickle.dump(model, open('MultiNB_model_89_accu.pkl', 'wb'))
pickle.dump(vectorizer, open('trigram_vectorizer.pkl', 'wb'))